In [1]:
import numpy as np
import scipy as sp
from importlib import reload
import neuraltda.topology2 as tp2
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload

import seaborn as sns
import pandas as pd
import pickle

import neuraltda.spectralAnalysis as sa
import neuraltda.simpComp as sc

/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
block_path = '/home/brad/krista/B1083/P03S03/'

bps = {'B1083': '/home/brad/krista/B1083/P03S03/', 'B1075': '/home/brad/krista/B1075/P01S03/', 'B1235': '/home/brad/krista/B1235/P02S01/'}


In [ ]:
#bin the data
winsize = 10.0
dtovr = winsize*0.5
comment = 'ForJSDivergence'
segment_info = [0,0]

bdfs = []
birds = ['B1083', 'B1235', 'B1075']
for bird in birds:
    block_path = bps[bird]
    bfdict = tp2.dag_bin(block_path, winsize, segment_info, cluster_group=['Good'], dt_overlap=dtovr, comment=comment)
    bdf = glob.glob(os.path.join(bfdict['raw'], '*.binned'))[0]
    print(bdf)
    bdfs.append([bird, bdf])

In [ ]:
# compute scgs.  
thresh = 6.0
cmnt = 'ForJSDivergence'

scgGenFiles = {}
for bdfj in bdfs:
    bdf = bdfj[1]
    scgGenFiles[bdfj[0]] = sa.computeChainGroups(block_path, bdf, thresh, comment=cmnt)

In [ ]:
scgf = scgGenFiles['B1083']

In [146]:

scgf = '/home/brad/krista/B1083/P03S03/scg/20170731T155520Z-10.0-5.0-10.0-ForJSDivergence.scg'



with open(scgf, 'rb') as f:
    scg = pickle.load(f)
    
print(scgf)

/home/brad/krista/B1083/P03S03/scg/20170731T155520Z-10.0-5.0-10.0-ForJSDivergence.scg


In [147]:
trials_A = scg['I_40k'][:10]
trials_B = scg['I_40k'][10:]

In [148]:
trials_A = {}
trials_B = {}

for stim in scg.keys():
    trials_A[stim] = scg[stim][:10]
    trials_B[stim] = scg[stim][10:]

In [149]:
# within stim
reload(sa)
d = 2
betas = np.linspace(0.001, 1, 20)
JS_save = {}
for stim in scg.keys():
    print(stim)
    JS_stim = np.zeros((10, len(betas)))
    for A1 in range(10):
        for B1 in range(1):
            print(A1)
            scgA = trials_A[stim][A1]
            scgB = trials_B[stim][A1]
            JS = []
            for beta in betas:
                print(beta)
                JS = sa.compute_JS_expanded(scgA, scgB, d, beta)
            JS_stim[A1, :] = np.array(JS)
    JS_save[stim] = JS_stim

I_40k
0
0.001
Computing Laplacians
Reconciling Laplacians
Computing Density Matrices


KeyboardInterrupt: 

In [6]:
scgA = scg['I_40k'][0]

In [141]:
reload(sc)

<module 'neuraltda.simpComp' from '/home/brad/code/NeuralTDA/neuraltda/simpComp.py'>

In [8]:
%time test = sc.boundaryOperatorMatrix(scgA, 3)

CPU times: user 1.43 s, sys: 12 ms, total: 1.44 s
Wall time: 1.44 s


In [136]:
%time test = sc.compute_laplacian(scgA, 3)

CPU times: user 6.55 s, sys: 444 ms, total: 7 s
Wall time: 4.1 s


In [60]:
%time vals = sp.linalg.eigh(test, eigvals_only=True)

CPU times: user 24.3 s, sys: 128 ms, total: 24.4 s
Wall time: 6.26 s


In [63]:
rhovals = np.exp(vals)/np.sum(np.exp(vals))
print(rhovals)

[  1.31184728e-13   1.89082249e-13   1.95358821e-13 ...,   3.18152726e-04
   2.21927089e-03   9.97031780e-01]


In [56]:
%time (a, b) = np.linalg.eig(test)

CPU times: user 1min 5s, sys: 844 ms, total: 1min 6s
Wall time: 21.6 s


In [13]:
b.shape

(4092, 4092)

In [15]:
sp.sparse.issparse(test)

False

In [45]:
test2 = sp.sparse.csc_matrix(test)

In [54]:
%time (a, b) = sp.sparse.linalg.eigs(test2, k=4000)

CPU times: user 9min 33s, sys: 3.41 s, total: 9min 36s
Wall time: 4min 11s


In [53]:
a

array([ 30.21193339+0.j,  24.10432946+0.j,  22.16192701+0.j,
        20.59173526+0.j,  20.34372557+0.j,  20.10387792+0.j,
        20.05560045+0.j,  19.95008931+0.j,  19.61775920+0.j,
        19.25892712+0.j,  19.12736796+0.j,  18.55431853+0.j,
        18.43094209+0.j,  18.31734750+0.j,  18.28900764+0.j,
        18.21355593+0.j,  18.19864393+0.j,  18.11212320+0.j,
        17.78412853+0.j,  17.59179204+0.j,  17.56892420+0.j,
        17.48789843+0.j,  17.31536470+0.j,  17.27242322+0.j,
        17.21743972+0.j,  17.16015287+0.j,  17.12082883+0.j,
        17.08279789+0.j,  17.03663965+0.j,  16.92482322+0.j,
        17.00000000+0.j,  17.00000000+0.j,  16.74364943+0.j,
        16.65167004+0.j,  16.46979301+0.j,  16.47064436+0.j,
        16.45121565+0.j,  16.42696590+0.j,  16.32703326+0.j,
        16.27263755+0.j,  16.26526649+0.j,  16.25812503+0.j,
        16.18834030+0.j,  16.13701193+0.j,  16.08030036+0.j,
        16.07610374+0.j,  16.05694003+0.j,  16.03461144+0.j,
        16.00664449+0.j,

In [41]:
%time test3 = sp.sparse.linalg.expm(test2)

/home/brad/.conda/envs/neuraltda-3/lib/python3.6/site-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


CPU times: user 8.39 s, sys: 8 ms, total: 8.4 s
Wall time: 7.78 s


In [26]:
%time (a, b) = np.linalg.eig(test)

CPU times: user 1min 30s, sys: 1.96 s, total: 1min 32s
Wall time: 38.5 s


In [38]:
%time test2 = sp.linalg.expm(k)

CPU times: user 1min 32s, sys: 13.3 s, total: 1min 45s
Wall time: 31.3 s


In [30]:
b[b < 0.0000001] = 0

In [37]:
k = np.diag(a)
k[k<0.000001]=0

In [42]:
test3

<4092x4092 sparse matrix of type '<class 'numpy.complex128'>'
	with 8183 stored elements in Compressed Sparse Column format>

In [43]:
sp.linalg.expm(np.eye(4))

array([[ 2.71828183,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  2.71828183,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  2.71828183,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  2.71828183]])

In [139]:
testE = sc.simplicialChainGroups([(1,2,3), (3,4,5), (4,5,6,7), (7,8,9)])
testE2 = sc.simplicialChainGroups([(1,2), (3,4,5), (4,5,6), (7,8), (8, 9,10, 11)])
testL = sc.compute_laplacian(testE, 1)
testL2 = sc.compute_laplacian(testE2, 1)
testR = sc.densityMatrix(testL, 1.0)
testR2 = sc.densityMatrix(testL2, 1.0)
testvals = sp.linalg.eigh(testR, eigvals_only=True)
print(testvals)

[ 0.00125604  0.00272912  0.00580973  0.01579249  0.01579249  0.01579249
  0.01579249  0.03243991  0.04292845  0.04292845  0.11669161  0.11669161
  0.15544634  0.41990876]


In [75]:

Lvals = sp.linalg.eigh(testL, eigvals_only=True)
rvals = np.exp(Lvals)/ np.sum(np.exp(Lvals))
print(rvals)

[ 0.00125604  0.00272912  0.00580973  0.01579249  0.01579249  0.01579249
  0.01579249  0.03243991  0.04292845  0.04292845  0.11669161  0.11669161
  0.15544634  0.41990876]


In [145]:
print(sa.compute_JS_expanded(testE, testE2, 1, 1.0))

Computing Laplacians
Reconciling Laplacians
Computing Density Matrices
Computing JS divergence
mlog
mlog
0.109088143705


In [150]:
testE = sc.simplicialChainGroups([(1,2,3), (3,4,5), (4,5,6,7), (7,8,9)])
testE2 = sc.simplicialChainGroups([(1,2), (3,4,5), (4,5,6), (7,8), (8, 9,10, 11)])
testL = sc.compute_laplacian(testE, 1)
testL2 = sc.compute_laplacian(testE2, 1)
(testL, testL2) = sc.reconcile_laplacians(testL, testL2)

testR = sc.densityMatrix(testL, 1.0)
testR2 = sc.densityMatrix(testL2, 1.0)

Lvals1 = sp.linalg.eigh(testL, eigvals_only=True)
rvals1 = np.real(np.exp(Lvals1)/ np.sum(np.exp(Lvals1)))

Lvals2 = sp.linalg.eigh(testL2, eigvals_only=True)
rvals2 = np.real(np.exp(Lvals2)/np.sum(np.exp(Lvals2)))

div = np.sum(np.multiply(rvals1, (np.log(rvals1) - np.log(rvals2))/ np.log(2.0)))
print(div)

0.441489285083


In [109]:
sc.KLdivergence(testR, testR2)

[ 0.24286875  0.0044483   0.08934642  0.08934642  0.01209172  0.01209172
  0.08934642  0.08934642  0.01209172  0.08934642  0.08934642  0.08934642
  0.08934642  0.00163644]
[ 0.41990876  0.15544634  0.03243991  0.00125604  0.00272912  0.11669161
  0.01579249  0.01579249  0.00580973  0.04292845  0.04292845  0.01579249
  0.11669161  0.01579249]


1.2841873372559884

In [102]:
v1 = sp.linalg.eigh(testR, eigvals_only=True)
print(rvals1 - v1)

[ -2.16840434e-19  -6.41847686e-17   1.38777878e-17   1.38777878e-17
  -6.93889390e-18   5.55111512e-17   4.16333634e-17  -2.77555756e-17
  -2.77555756e-17  -2.77555756e-17  -2.77555756e-17  -2.77555756e-17
  -2.77555756e-17   8.32667268e-17]


In [101]:
v2 = sp.linalg.eigh(testR2, eigvals_only=True)
print(rvals2-v2)

[ -3.25260652e-17  -3.46944695e-17   1.57859836e-16   5.55111512e-17
   1.38777878e-17   2.08166817e-17  -3.46944695e-18   3.46944695e-17
  -8.32667268e-17  -1.04083409e-16   3.05311332e-16   1.94289029e-16
  -4.16333634e-16  -2.22044605e-16]


In [103]:
np.sum(np.multiply(rvals1, (np.log(rvals1) - np.log(rvals2))/np.log(2.0)))

0.44148928508341312

In [165]:
reload(sc)


<module 'neuraltda.simpComp' from '/home/brad/code/NeuralTDA/neuraltda/simpComp.py'>

In [167]:
sc.KLdivergence(testR, testR2)

1.2841873372559884

In [118]:
np.log(np.trace(sp.linalg.expm(testL)))

6.4152341175112468

In [114]:
np.exp(np.trace(testL))

1.2851600114359308e+19

In [115]:
np.trace(testL)

44.0

In [117]:
np.sum(np.exp(Lvals1))

611.08380949298146

In [121]:
def KL_div2(L1, L2, beta):
    
    Lvals1 = sp.linalg.eigh(L1, eigvals_only=True)
    Lvals2 = sp.linalg.eigh(L2, eigvals_only=True)
    
    C1 = np.sum(np.exp(beta*Lvals1))
    C2 = np.sum(np.exp(beta*Lvals2))
    
    K = (beta * (L1 - L2) + (np.log(C2/C1))) / np.log(2.0)
    rho = sp.linalg.expm(beta*L1) / C1
    
    return np.trace(np.dot(rho, K))

In [122]:
KL_div2(testL, testL2, 1.0)

1.3729457474565299

In [127]:
testK = (sp.linalg.logm(testR) - sp.linalg.logm(testR2)) / np.log(2)

In [129]:
beta = 1.0
Lvals1 = sp.linalg.eigh(testL, eigvals_only=True)
Lvals2 = sp.linalg.eigh(testL2, eigvals_only=True)
    
C1 = np.sum(np.exp(beta*Lvals1))
C2 = np.sum(np.exp(beta*Lvals2))
    
K = (beta * (testL - testL2) + (np.log(C2/C1))) / np.log(2.0)

In [152]:
np.amax(np.abs(testR*testR2 - testR2*testR))

0.0

In [157]:
np.amax(np.abs(np.dot(testR, testR2) - np.dot(testR2, testR)))

0.0096078701681666778

In [158]:
np.amax(np.abs(np.dot(testL, testL2) - np.dot(testL2, testL)))

3.0

In [163]:
r, w = np.linalg.eig(testR)
s, w = np.linalg.eig(testR2)

In [164]:
print(r)
print(s)

[ 0.24286875  0.0044483   0.08934642  0.08934642  0.01209172  0.01209172
  0.08934642  0.08934642  0.01209172  0.08934642  0.08934642  0.08934642
  0.08934642  0.00163644]
[ 0.41990876  0.15544634  0.03243991  0.00125604  0.00272912  0.11669161
  0.01579249  0.01579249  0.00580973  0.04292845  0.04292845  0.01579249
  0.11669161  0.01579249]


In [161]:
testL2

array([[ 3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  3.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  3., -1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0., -1., -1.,  3.,  0.,  0., -1., -1.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0., -1., -1.,  0.,  3.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  0., -1., -1.,
         0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,
         0.],
       [ 0.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  4.,  0., -1., -1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  4., -1., -1.,
         0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 

In [168]:
sc.Likelihood(testR, testR2)

0.03161348629784292

In [193]:
r, w1 = np.linalg.eig(testR)
s, w2 = np.linalg.eig(testR2)

In [195]:
Pbar = np.dot(w2, w1.T).T

In [181]:
np.diag(k**2)

array([  1.95873229e-01,   8.54575026e-02,   3.63681310e-04,
         5.71985873e-03,   2.09786457e-01,   1.99178706e-02,
         4.10973162e-03,   3.64799567e-01,   0.00000000e+00,
         1.08316093e-31,   9.30597441e-02,   0.00000000e+00,
         0.00000000e+00,   1.00000000e+00])

In [205]:
rho = testR
sigma = testR2
sigmaprime = np.dot(Pbar, np.dot(testR2, Pbar.T))

sigmaprime2 = np.dot(w1, np.dot(np.diag(s), w1.T))

In [197]:
sc.KLdivergence_matrixlog(rho, sigmaprime)

0.47212506396792708

In [198]:
sc.KLdivergence(rho, sigma)

1.2841873372559884

In [206]:
sc.KLdivergence_matrixlog(rho, sigmaprime2)

0.47193116845076227

In [204]:
t, x = np.linalg.eig(sigmaprime)
print(t)

[ 0.00580973  0.41990876  0.15544634  0.11672107  0.00085693  0.00269966
  0.01140354  0.01516292  0.03749457  0.04383053  0.04202637  0.01579249
  0.11669161  0.01579249]


In [200]:
s

array([ 0.41990876,  0.15544634,  0.03243991,  0.00125604,  0.00272912,
        0.11669161,  0.01579249,  0.01579249,  0.00580973,  0.04292845,
        0.04292845,  0.01579249,  0.11669161,  0.01579249])

In [207]:
t, x = np.linalg.eig(sigmaprime2)
print(t)

[ 0.41990876  0.15544634  0.03785214  0.01545193  0.01111928  0.00085759
  0.11672107  0.00269966  0.00580973  0.04292845  0.04292845  0.01579249
  0.11669161  0.01579249]


In [189]:
Drho = np.diag(r)

In [208]:
Pbar

array([[  0.00000000e+00,   7.71624497e-17,  -5.69599232e-16,
         -5.77350269e-01,   5.77350269e-01,   0.00000000e+00,
         -2.88675135e-01,  -2.88675135e-01,   2.88675135e-01,
          2.88675135e-01,  -4.55629494e-16,  -2.33624709e-17,
         -1.13261000e-15,   0.00000000e+00],
       [  0.00000000e+00,   2.95179495e-01,   1.88701996e-01,
         -1.14622934e-01,   1.67638439e-01,   0.00000000e+00,
          1.49559755e-02,  -1.38544195e-01,  -2.67305397e-01,
         -4.20805568e-01,  -1.53500171e-01,  -8.47134740e-01,
          1.37581210e-01,   0.00000000e+00],
       [  0.00000000e+00,  -4.05498471e-01,   5.67782446e-01,
          3.42436167e-01,  -1.61212372e-01,   0.00000000e+00,
         -2.04776231e-01,  -3.05367416e-01,   2.98872308e-01,
          1.98281122e-01,  -1.00591185e-01,  -1.22209231e-01,
         -1.25082761e-01,   0.00000000e+00],
       [  0.00000000e+00,   1.39247329e-15,  -6.19499579e-17,
         -3.29114104e-16,  -5.12497194e-16,   0.00000000e+0